In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = './model/qwen2p5_7b'
lora_path = './output/qwen2p5_7b/checkpoint-702' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)
model.merge_and_unload()  # LoRA 权重直接写入 base_model


`torch_dtype` is deprecated! Use `dtype` instead!
/root/huanhuanchat/.venv/lib/python3.11/site-packages/accelerate/utils/modeling.py:1582: UserWarning: Current model requires 128 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/root/huanhuanchat/.venv/lib/python3.11/site-packages/accelerate/utils/modeling.py:1582: UserWarning: Current model requires 256 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), eps=1e-06)
    (rotary_emb):

In [3]:
merged_model_dir = "./qwen2.5_7B_lora_merged"
model.save_pretrained(merged_model_dir)
tokenizer.save_pretrained(merged_model_dir)

('./qwen2.5_7B_lora_merged/tokenizer_config.json',
 './qwen2.5_7B_lora_merged/special_tokens_map.json',
 './qwen2.5_7B_lora_merged/chat_template.jinja',
 './qwen2.5_7B_lora_merged/vocab.json',
 './qwen2.5_7B_lora_merged/merges.txt',
 './qwen2.5_7B_lora_merged/added_tokens.json',
 './qwen2.5_7B_lora_merged/tokenizer.json')

In [4]:
from modelscope.hub.api import HubApi

api = HubApi()
api.login()

(None, None)

In [5]:
from modelscope.hub.constants import Licenses, ModelVisibility

owner_name = 'nev8rz'
model_name = 'huanhuanchat_base_qwen2.5_7B'
model_id = f"{owner_name}/{model_name}"

api.create_model(
    model_id,
    visibility=ModelVisibility.PUBLIC,
    license=Licenses.APACHE_V2,
    chinese_name="嬛嬛"
)

'https://www.modelscope.cn/models/nev8rz/huanhuanchat_base_qwen2.5_7B'

In [6]:
api.upload_folder(
    repo_id=f"{owner_name}/{model_name}",
    folder_path='./qwen2.5_7B_lora_merged/',
    commit_message='聪明一点的嬛嬛',
)

2025-09-26 22:46:28,257 - modelscope - INFO - Preparing files to upload ...
2025-09-26 22:46:28,270 - modelscope - INFO - Prepared 10 files for upload.
2025-09-26 22:46:28,270 - modelscope - INFO - Checking 10 files to upload ...
2025-09-26 22:46:28,508 - modelscope - WARNING - Repo nev8rz/huanhuanchat_base_qwen2.5_7B already exists, got repo url: https://www.modelscope.cn/models/nev8rz/huanhuanchat_base_qwen2.5_7B


Processing 10 items:   0%|          | 0.00/10.0 [00:00<?, ?it/s]

Committing 10 files in 1 batch(es) of size 512.


[Committing batches] :   0%|          | 0/1 [00:00<?, ?it/s]

2025-09-26 22:46:31,286 - modelscope - INFO - Commit succeeded: https://www.modelscope.cn/api/v1/repos/models/nev8rz/huanhuanchat_base_qwen2.5_7B/commit/master


CommitInfo(commit_url='https://www.modelscope.cn/api/v1/repos/models/nev8rz/huanhuanchat_base_qwen2.5_7B/commit/master', commit_message='聪明一点的嬛嬛 (batch 1/1)', commit_description='Uploading files', oid='')